In [67]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [68]:

def get_auc(test_question):
    label_encoder = LabelEncoder()
    test_question['label_encoded'] = label_encoder.fit_transform(test_question['label'])
    test_question['extracted_answer_encoded'] = label_encoder.transform(test_question['extracted_answer'])
    auc_score = roc_auc_score(test_question['label_encoded'], test_question['extracted_answer_encoded'])
    return auc_score

def extract_answer(text):
    pattern = r"(True|False|Don't know)"
    matches = re.findall(pattern, text)
    return matches



In [69]:
PROMPT_RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_prompt_based_response.csv"
RAG_RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_node_retrieval_rag_based_response.csv"


In [70]:
response_transform = {
    "True" : True,
    "False" : False,
    "Don't know" : "Don't know"
}


In [71]:
prompt_response_df = pd.read_csv(PROMPT_RESPONSE_PATH)

prompt_response_df.loc[:, 'extracted_answer'] = prompt_response_df['llm_answer'].apply(extract_answer)
prompt_response_df.loc[:, "answer_count"] = prompt_response_df.extracted_answer.apply(lambda x:len(x))

prompt_response_df_multiple_answers = prompt_response_df[prompt_response_df.answer_count > 1]
prompt_response_df_single_answer = prompt_response_df.drop(prompt_response_df_multiple_answers.index)
prompt_response_df_single_answer.drop("answer_count", axis=1, inplace=True)


prompt_response_df_multiple_answers_ = []
for index, row in prompt_response_df_multiple_answers.iterrows():
    if row["extracted_answer"][0] == row["extracted_answer"][1]:
        prompt_response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], row["extracted_answer"][0]))
    else:
        prompt_response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], "Don't know"))

prompt_response_df_multiple_answers_ = pd.DataFrame(prompt_response_df_multiple_answers_, columns=["question", "label", "llm_answer", "extracted_answer"])

prompt_response_df_final = pd.concat([prompt_response_df_single_answer, prompt_response_df_multiple_answers_], ignore_index=True)
prompt_response_df_final = prompt_response_df_final.explode("extracted_answer")
prompt_response_df_final['extracted_answer'].fillna("Don't know", inplace=True)

prompt_response_df_final.extracted_answer = prompt_response_df_final.extracted_answer.apply(lambda x:response_transform[x])
prompt_response_df_certain = prompt_response_df_final[prompt_response_df_final.extracted_answer != "Don't know"]
prompt_response_df_uncertain = prompt_response_df_final[prompt_response_df_final.extracted_answer == "Don't know"]

label_encoder = LabelEncoder()
prompt_response_df_certain['label_encoded'] = label_encoder.fit_transform(prompt_response_df_certain['label'])
prompt_response_df_certain['extracted_answer_encoded'] = label_encoder.transform(prompt_response_df_certain['extracted_answer'])
prompt_response_df_correct_response = prompt_response_df_certain[prompt_response_df_certain.label_encoded == prompt_response_df_certain.extracted_answer_encoded]
prompt_response_df_incorrect_response = prompt_response_df_certain[prompt_response_df_certain.label_encoded != prompt_response_df_certain.extracted_answer_encoded]

prompt_response_df_certain.head()


/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_14215/1090471572.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompt_response_df_certain['label_encoded'] = label_encoder.fit_transform(prompt_response_df_certain['label'])
/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_14215/1090471572.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompt_response_df_certain['extracted_answer_encoded'] = label_encoder.transform(prompt_response_df_certain['extracted_answer'])


,question,label,llm_answer,extracted_answer,label_encoded,extracted_answer_encoded
0,enhanced S-cone syndrome is not a vitreoretina...,False,"{\n""answer"": ""False""\n}\n\nEnhanced S-cone s...",False,0,0
1,metronidazole treats crohn's disease,True,"{\n""answer"": ""False""\n}\n\nMetronidazole is ...",False,1,0
2,KLEEFSTRA SYNDROME 1 is not associated with Ge...,False,"{\n ""answer"": ""False""\n}",False,0,0
3,STARGARDT DISEASE 1 (disorder) is not associat...,False,"{\n ""answer"": ""False""\n}",False,0,0
4,Juvenile polyposis syndrome associates Gene SMAD4,True,"{\n""answer"": ""True""\n}",True,1,1


In [72]:
rag_response_df = pd.read_csv(RAG_RESPONSE_PATH)

rag_response_df.loc[:, 'extracted_answer'] = rag_response_df['llm_answer'].apply(extract_answer)
rag_response_df.loc[:, "answer_count"] = rag_response_df.extracted_answer.apply(lambda x:len(x))

rag_response_df_multiple_answers = rag_response_df[rag_response_df.answer_count > 1]
rag_response_df_single_answer = rag_response_df.drop(rag_response_df_multiple_answers.index)
rag_response_df_single_answer.drop("answer_count", axis=1, inplace=True)


rag_response_df_multiple_answers_ = []
for index, row in rag_response_df_multiple_answers.iterrows():
    if row["extracted_answer"][0] == row["extracted_answer"][1]:
        rag_response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], row["extracted_answer"][0]))
    else:
        rag_response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], "Don't know"))

rag_response_df_multiple_answers_ = pd.DataFrame(rag_response_df_multiple_answers_, columns=["question", "label", "llm_answer", "extracted_answer"])

rag_response_df_final = pd.concat([rag_response_df_single_answer, rag_response_df_multiple_answers_], ignore_index=True)
rag_response_df_final = rag_response_df_final.explode("extracted_answer")
rag_response_df_final['extracted_answer'].fillna("Don't know", inplace=True)

rag_response_df_final.extracted_answer = rag_response_df_final.extracted_answer.apply(lambda x:response_transform[x])
rag_response_df_certain = rag_response_df_final[rag_response_df_final.extracted_answer != "Don't know"]
rag_response_df_uncertain = rag_response_df_final[rag_response_df_final.extracted_answer == "Don't know"]

label_encoder = LabelEncoder()
rag_response_df_certain['label_encoded'] = label_encoder.fit_transform(rag_response_df_certain['label'])
rag_response_df_certain['extracted_answer_encoded'] = label_encoder.transform(rag_response_df_certain['extracted_answer'])
rag_response_df_correct_response = rag_response_df_certain[rag_response_df_certain.label_encoded == rag_response_df_certain.extracted_answer_encoded]
rag_response_df_incorrect_response = rag_response_df_certain[rag_response_df_certain.label_encoded != rag_response_df_certain.extracted_answer_encoded]

rag_response_df_certain.head()


/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_14215/3915019513.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rag_response_df_certain['label_encoded'] = label_encoder.fit_transform(rag_response_df_certain['label'])
/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_14215/3915019513.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rag_response_df_certain['extracted_answer_encoded'] = label_encoder.transform(rag_response_df_certain['extracted_answer'])


,question,label,llm_answer,extracted_answer,label_encoded,extracted_answer_encoded
0,enhanced S-cone syndrome is not a vitreoretina...,False,"{\n""answer"": ""True""\n}",True,0,1
1,metronidazole treats crohn's disease,True,"{\n""answer"": ""True""\n}",True,1,1
2,KLEEFSTRA SYNDROME 1 is not associated with Ge...,False,"{\n""answer"": ""False""\n}",False,0,0
3,STARGARDT DISEASE 1 (disorder) is not associat...,False,"{\n""answer"": ""False""\n}",False,0,0
4,Juvenile polyposis syndrome associates Gene SMAD4,True,"{\n""answer"": ""True""\n}",True,1,1


In [73]:
rag_response_df_uncertain[rag_response_df_uncertain.question.isin(prompt_response_df_correct_response.question)]


,question,label,llm_answer,extracted_answer
265,Lecithin Acyltransferase Deficiency associates...,True,"{\n""answer"": ""Don't know""\n}",Don't know
365,Variant rs2476601 associates Rheumatoid Arthritis,True,"{\n""answer"": ""Don't know""\n}",Don't know
425,Septo-Optic Dysplasia associates Gene HESX1,True,Based on the information provided in the con...,Don't know
443,Hidrotic Ectodermal Dysplasia associates Gene ...,True,"Based on the information provided, the answe...",Don't know


In [76]:
rag_response_df_incorrect_response[rag_response_df_incorrect_response.question.isin(prompt_response_df_correct_response.question)]


,question,label,llm_answer,extracted_answer,label_encoded,extracted_answer_encoded
0,enhanced S-cone syndrome is not a vitreoretina...,False,"{\n""answer"": ""True""\n}",True,0,1
37,"Crigler Najjar syndrome, type 1 associates Gen...",True,"{\n""answer"": ""False""\n}",False,1,0
73,emicizumab treats hemophilia a,True,"Sure! Based on the information provided, her...",False,1,0
91,vitelliform macular dystrophy is not a macular...,False,"{\n""answer"": ""True""\n}",True,0,1
114,Bloom syndrome is not a autosomal recessive di...,False,"{\n""answer"": ""True""\n}",True,0,1
123,Ichthyosis linearis circumflexa associates Gen...,True,"{\n""answer"": ""False""\n}\n\nIchthyosis linear...",False,1,0
137,Okihiro Syndrome associates Gene SALL4,True,"Based on the information provided, the answe...",False,1,0
138,Aniridia is not associated with Gene PAX6,False,"Sure, I'd be happy to help!\n\nAfter reviewi...",True,0,1
148,Johanson-Blizzard syndrome associates Gene UBR1,True,"{\n ""answer"": ""False""\n}",False,1,0
164,alpha-galactosidase treats fabry disease,True,"{\n ""answer"": ""False""\n}\n\n alpha-galactos...",False,1,0


In [81]:
prompt_response_df_incorrect_response.shape


(60, 6)

In [77]:
rag_response_df_incorrect_response[rag_response_df_incorrect_response.question.isin(prompt_response_df_correct_response.question)].question.values


array(['enhanced S-cone syndrome is not a vitreoretinal degeneration',
       'Crigler Najjar syndrome, type 1 associates Gene UGT1A1',
       'emicizumab treats hemophilia a',
       'vitelliform macular dystrophy is not a macular degeneration',
       'Bloom syndrome is not a autosomal recessive disease',
       'Ichthyosis linearis circumflexa associates Gene SPINK5',
       'Okihiro Syndrome associates Gene SALL4',
       'Aniridia is not associated with Gene PAX6',
       'Johanson-Blizzard syndrome associates Gene UBR1',
       'alpha-galactosidase treats fabry disease',
       'Piebaldism associates Gene KIT',
       'Papillorenal syndrome associates Gene PAX2',
       'Chronic Infantile Neurological, Cutaneous, and Articular Syndrome associates Gene NLRP3',
       'congenital contractural arachnodactyly is not a congenital nervous system disorder',
       'Hyperkalemic periodic paralysis is not associated with Gene SCN4A',
       'factor viia treats hemophilia a',
       'alpha

In [78]:
rag_response_df_incorrect_response[rag_response_df_incorrect_response.question.isin(prompt_response_df_correct_response.question)].llm_answer.values


array(['  {\n"answer": "True"\n}', '  {\n"answer": "False"\n}',
       '  Sure! Based on the information provided, here is my answer in the requested JSON format:\n\n{\n"answer": "False"\n}\n\nEmicizumab does not treat hemophilia A, as it is indicated for the treatment of hemophilia B.',
       '  {\n"answer": "True"\n}', '  {\n"answer": "True"\n}',
       '  {\n"answer": "False"\n}\n\nIchthyosis linearis circumflexa does not associate with the Gene SPINK5.',
       '  Based on the information provided, the answer is:\n\n{\n"answer": "False"\n}\n\nOkihiro Syndrome is not associated with the gene SALL4.',
       '  Sure, I\'d be happy to help!\n\nAfter reviewing the context, I can confirm that Aniridia is associated with the PAX6 gene. Therefore, the answer is:\n\n{\n"answer": "True"\n}\n\nPAX6 is a critical gene that plays a crucial role in the development of the eye, and mutations in this gene have been linked to Aniridia, a condition characterized by a partial or complete absence of 

In [66]:
rag_response_df_uncertain[rag_response_df_uncertain.question.isin(prompt_response_df_correct_response.question)].question.values


array(['KLEEFSTRA SYNDROME 1 is not associated with Gene EHMT1',
       'Meleda Disease is not associated with Gene SLURP1',
       'Limb-girdle muscular dystrophy type 2H associates Gene TRIM32',
       'Arthrogryposis, renal dysfunction, and cholestasis 1 associates Gene VPS33B',
       'X-linked hydrocephalus syndrome associates Gene L1CAM',
       'Cardiomyopathy, Familial Idiopathic associates Gene LMNA',
       'piebaldism is a autosomal dominant disease',
       'Senter syndrome associates Gene GJB2',
       'MEGALENCEPHALIC LEUKOENCEPHALOPATHY WITH SUBCORTICAL CYSTS associates Gene MLC1',
       'Shprintzen syndrome associates Gene TBX1',
       'HEMOCHROMATOSIS, TYPE 4 associates Gene SLC40A1',
       'Ichthyosis linearis circumflexa associates Gene SPINK5',
       'Congenital chloride diarrhea associates Gene SLC26A3',
       'Congenital Nonbullous Ichthyosiform Erythroderma associates Gene TGM1',
       'Carney complex is a autosomal dominant disease',
       'MASA SYNDROME 